In [2]:
import os
import pandas as pd
import glob
from tqdm import tqdm
from scipy.io import wavfile
from scipy import signal
import numpy as np
import soundfile as sf
os.chdir('../src/')
import config
import matplotlib.pyplot as plt
from tqdm import tqdm
import IPython.display as ipd

In [3]:
gt_train = pd.read_csv(config.DATA_FOLDER + 'gt_train_stats.csv')
gt_test = pd.read_csv(config.DATA_FOLDER + 'gt_test_stats.csv')

In [4]:
nn_folder = 'stacker_xgb_11'
sub = {}
for flag in ['train', 'test']:
    _ = pd.read_csv(config.OUTPUT_FOLDER + '%s/sub_%s.csv'%(nn_folder, flag))
    _.rename(columns = {'target' : 'pred_target'}, inplace = True)
    _['confidence'] = _[config.mapping_dict.keys()].max(axis = 1)
    gt = pd.read_csv(config.DATA_FOLDER + 'gt_%s_stats.csv'%flag)
    _ = _.merge(gt, on = 'id')
    sub[flag] = _
    assert len(_) == len(gt)


In [10]:
delta = 0
low_volume_ids = gt_test[(gt_test['stat_max_audio'] <=delta ) & \
                         (gt_test['stat_min_audio'] >= -delta)].sort_values(['stat_min_audio'])['id'].tolist()
print(len(low_volume_ids))

3709


In [9]:
delta = 3
gt_train[(gt_train['stat_max_audio'] <=delta ) & (gt_train['stat_min_audio'] >= delta)].sort_values(['stat_min_audio'])

,id,path,person_id,target,stat_max_audio,stat_min_audio,stat_mean_audio,stat_std_audio


In [11]:
submission = sub['test'][['id', 'pred_target']].copy() 
submission.loc[submission['id'].isin(low_volume_ids), 'pred_target'] = 'silence'
submission['label'] = submission['pred_target']
submission['label'] = submission['label'].apply(lambda x: x if x in config.allowed_train_labels else 'unknown')
submission['fname'] = submission['id'] + '.wav'

In [12]:
submission[['fname', 'label']].to_csv('stacker_xgb_11_volume_delta_0_to_silence.csv', index = False)

In [63]:
len(gt_test[gt_test['stat_max_audio'] <= 200]) / float(len(gt_test))

0.06452080889124374

In [58]:
ipd.Audio(res_df[res_df['stat_max_audio'] <= 259]['path'].values[20])